# Taller 1: Aplicación de RNNs al Modelamiento de Lenguaje en Español (LSTM/BiLSTM)

**Curso:** PLN
**Objetivo:** Experimentar con modelos de Redes Neuronales Recurrentes (RNNs), específicamente LSTM y BiLSTM, para el modelado del lenguaje en español. 
**Autor:** Herney Eduardo Quintero Trochez  
**Fecha:** 2025  
**Universidad:** Universidad Del Valle  
**Curso:** Procesamiento de Lenguaje Natural (PLN) - Taller 1
## Componentes implementados:
1. Carga del Dataset (`spanish_billion_words_clean`)
2. Tokenización y Creación del Vocabulario
3. Creación del Conjunto de Entrenamiento (X, Y)
4. Padding y Truncado (MAX_LEN ≤ 50)
5. División del Conjunto (Train/Test 80%/20%)
6. Construcción del Modelo LSTM/BiLSTM
7. Entrenamiento con Early Stopping
8. Cálculo de la Perplejidad
9. Predicción de la Próxima Palabra

## 0. Configuración de Parámetros Globales

Esta sección permite modificar fácilmente todos los parámetros del modelo para experimentar.

In [ ]:
# Dataset configuration
DATASET_NAME = "jhonparra18/spanish_billion_words_clean"
DATASET_SPLIT = "train"
DATASET_STREAMING = True
DATASET_TAKE = 5000  # Número de ejemplos a tomar del dataset
MIN_WORDS_PER_SENTENCE = 4  # Filtro: oraciones con al menos N palabras
OOV_TOKEN = "<OOV>"  # Token para palabras fuera del vocabulario

# Model architecture parameters
EMBEDDING_DIM = 50  # Dimensión del embedding de palabras
LSTM_UNITS = 64  # Número de unidades en las capas LSTM
DENSE_UNITS = 64  # Número de unidades en la capa densa intermedia
USE_BIDIRECTIONAL = True  # Usar BiLSTM en lugar de LSTM unidireccional

# Training parameters
EPOCHS = 30  # Número de épocas de entrenamiento
BATCH_SIZE = 8  # Tamaño del batch
VALIDATION_SPLIT = 0.2  # Porcentaje de datos para validación
LEARNING_RATE = 0.001  # Tasa de aprendizaje

# Sequence processing
PADDING_TYPE = 'pre'  # Tipo de padding: 'pre' o 'post'

# Output parameters
VERBOSE_TRAINING = 1  # Nivel de verbosidad durante entrenamiento
VERBOSE_PREDICTION = 0  # Nivel de verbosidad durante predicción

print(f"Configuración cargada:")
print(f"- Modelo: {'BiLSTM' if USE_BIDIRECTIONAL else 'LSTM'}")
print(f"- Dataset: {DATASET_TAKE:,} muestras de {DATASET_NAME}")
print(f"- Arquitectura: {EMBEDDING_DIM}D embedding → LSTM({LSTM_UNITS}) → Dense({DENSE_UNITS})")
print(f"- Entrenamiento: {EPOCHS} epochs, batch_size={BATCH_SIZE}")

## 1. Importación de Librerías

Se importan todas las librerías necesarias para el proyecto.

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from datasets import load_dataset
import math

print("Librerías importadas exitosamente!")
print(f"TensorFlow version: {tf.__version__}")

## 2. Carga del Dataset

Se carga el dataset `spanish_billion_words_clean` de Hugging Face y se filtran oraciones muy cortas.

In [ ]:
print("Cargando dataset...")
dataset = load_dataset(DATASET_NAME, split=DATASET_SPLIT, streaming=DATASET_STREAMING).take(DATASET_TAKE)
sentences = [example['text'] for example in dataset if len(example['text'].split()) >= MIN_WORDS_PER_SENTENCE]

print(f"Total de oraciones cargadas: {len(sentences)}")
print("\nEjemplos de oraciones:")
for i, sentence in enumerate(sentences[:5]):
    print(f"{i+1}. {sentence}")

## 3. Tokenización y Creación del Vocabulario

Se Tokenizan las oraciones y se crea el vocabulario con mapeo palabra→índice.

In [ ]:
tokenizer = Tokenizer(oov_token=OOV_TOKEN)  # Token para palabras desconocidas
tokenizer.fit_on_texts(sentences)

# Convertir frases a secuencias numéricas
sequences = tokenizer.texts_to_sequences(sentences)

# Tamaño del vocabulario
vocab_size = len(tokenizer.word_index) + 1  # +1 por el padding (índice 0)
print(f"Vocabulario: {vocab_size} palabras")

print("\nPrimeras 10 palabras del vocabulario:")
for i, (word, idx) in enumerate(list(tokenizer.word_index.items())[:10]):
    print(f"{word} → {idx}")

print(f"\nEjemplo de tokenización:")
example_sentence = sentences[0]
example_sequence = sequences[0]
print(f"Oración: {example_sentence}")
print(f"Secuencia: {example_sequence}")

## 4. Creación del Conjunto de Entrenamiento (X, Y)

Para cada secuencia `[w1, w2, ..., wn]`, se crean pares:
- `(w1) → w2`
- `(w1, w2) → w3`
- `...`
- `(w1, ..., wn-1) → wn`

In [ ]:
X = []  # Secuencias de entrada: [palabra1], [palabra1, palabra2], ...
y = []  # Palabra siguiente (objetivo)

for seq in sequences:
    for i in range(len(seq) - 1):
        X.append(seq[:i+1])    # Desde el inicio hasta la palabra actual
        y.append(seq[i+1])     # La siguiente palabra

print(f"Total de secuencias de entrenamiento generadas: {len(X)}")
print("\nEjemplos de secuencias X → y:")
for i in range(5):
    # Convertir índices a palabras para mostrar
    x_words = [tokenizer.index_word.get(idx, '<UNK>') for idx in X[i]]
    y_word = tokenizer.index_word.get(y[i], '<UNK>')
    print(f"{x_words} → {y_word}")

## 5. Padding y Truncado (MAX_LEN ≤ 50)

Se Aplica padding a las secuencias y se trunca si la longitud máxima excede 50.

In [ ]:
# Longitud máxima de las secuencias (con restricción MAX_LEN <= 50)
raw_max_length = max([len(seq) for seq in X])
MAX_LEN = min(50, raw_max_length)  # Aplicar restricción de máximo 50
print(f"Longitud máxima encontrada: {raw_max_length}")
print(f"MAX_LEN aplicado (truncado a 50): {MAX_LEN}")

# Aplicar padding y truncado a las secuencias de entrada
X_padded = pad_sequences(X, maxlen=MAX_LEN, padding=PADDING_TYPE, truncating='pre')
y = np.array(y)  # Etiquetas: ya es un array 1D

print(f"\nForma de X_padded: {X_padded.shape}")  # (número de muestras, MAX_LEN)
print(f"Forma de y: {y.shape}")
print("\nEjemplo de X_padded (primeras 3 secuencias):")
print(X_padded[:3])
print("\nEjemplo de y (primeras 10 etiquetas):")
print(y[:10])

## 6. División del Conjunto (Train/Test: 80%/20%)

Se dividen los datos en conjuntos de entrenamiento y prueba usando `train_test_split`.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X_padded, y, 
    test_size=0.2, 
    random_state=42, 
    shuffle=True
)

print(f"División del conjunto:")
print(f"X_train: {X_train.shape}, y_train: {y_train.shape}")
print(f"X_test: {X_test.shape}, y_test: {y_test.shape}")
print(f"\nPorcentajes:")
print(f"Entrenamiento: {len(X_train)/(len(X_train)+len(X_test))*100:.1f}%")
print(f"Prueba: {len(X_test)/(len(X_train)+len(X_test))*100:.1f}%")


## 7. Construcción del Modelo LSTM/BiLSTM

Se crea el modelo con arquitectura configurable (LSTM o BiLSTM) y se compila con `sparse_categorical_crossentropy`.

In [ ]:
def create_model(vocab_size, max_length):
    """
    Crea un modelo de lenguaje usando BiLSTM o LSTM según configuración.
    
    Args:
        vocab_size (int): Tamaño del vocabulario
        max_length (int): Longitud máxima de las secuencias
    
    Returns:
        tensorflow.keras.Model: Modelo compilado
    """
    model = Sequential([
        Embedding(vocab_size, EMBEDDING_DIM, input_length=max_length, name='embedding'),
        # Usar BiLSTM si está habilitado, sino LSTM unidireccional
        Bidirectional(LSTM(LSTM_UNITS, name='lstm'), name='bidirectional_lstm') if USE_BIDIRECTIONAL 
        else LSTM(LSTM_UNITS, name='lstm'),
        Dense(DENSE_UNITS, activation='relu', name='dense_hidden'),
        Dense(vocab_size, activation='softmax', name='output')  # Probabilidades para cada palabra
    ])
    
    # Configurar optimizador con tasa de aprendizaje personalizada
    optimizer = Adam(learning_rate=LEARNING_RATE)
    
    # Usar sparse_categorical_crossentropy (NO necesita one-hot encoding)
    model.compile(
        loss='sparse_categorical_crossentropy',
        optimizer=optimizer,
        metrics=['accuracy', 'sparse_top_k_categorical_accuracy']
    )
    
    return model

model = create_model(vocab_size, MAX_LEN)
model.summary()

## 8. Entrenamiento del Modelo con Early Stopping

Se entrena el modelo con Early Stopping para prevenir overfitting.

In [ ]:
# Configurar Early Stopping
early_stopping = EarlyStopping(
    monitor='val_loss',     # Métrica a monitorear
    patience=5,             # Número de épocas sin mejora antes de parar
    restore_best_weights=True,  # Restaurar los mejores pesos
    verbose=1               # Mostrar información cuando se pare
)

print(f"Entrenando el modelo {'BiLSTM' if USE_BIDIRECTIONAL else 'LSTM'}...")
print(f"Arquitectura: {EMBEDDING_DIM}D embedding → {'Bi-' if USE_BIDIRECTIONAL else ''}LSTM({LSTM_UNITS}) → Dense({DENSE_UNITS}) → Softmax({vocab_size})")
print(f"Parámetros de entrenamiento: {EPOCHS} epochs, batch_size={BATCH_SIZE}, lr={LEARNING_RATE}")
print("Early Stopping habilitado: val_loss con patience=5")

history = model.fit(
    X_train, y_train,  # Usar conjuntos de entrenamiento divididos
    epochs=EPOCHS,
    verbose=VERBOSE_TRAINING,
    batch_size=BATCH_SIZE,
    validation_split=VALIDATION_SPLIT,  # Usar datos de entrenamiento para validación
    shuffle=True,
    callbacks=[early_stopping]  # Agregar Early Stopping
)

## 9. Cálculo de la Perplejidad

Se immplementan las funciones para calcular la perplejidad.

**Fórmula:** `perplexity = exp(-average_log_likelihood)`

In [ ]:
def calculate_perplexity(model, X_test, y_test):
    """
    Calcula la perplejidad del modelo en el conjunto de prueba.
    
    La perplejidad es una medida de qué tan bien un modelo de probabilidad 
    predice una muestra. Valores más bajos indican mejor rendimiento.
    
    Fórmula: perplexity = exp(average_negative_log_likelihood)
    
    Args:
        model: Modelo entrenado
        X_test: Conjunto de prueba de entrada
        y_test: Conjunto de prueba de salida (etiquetas verdaderas)
    
    Returns:
        float: Valor de perplejidad
    """
    # Obtener probabilidades predichas para el conjunto de prueba
    predictions = model.predict(X_test, verbose=0)
    
    # Calcular log-likelihood promedio
    log_likelihoods = []
    for i in range(len(y_test)):
        true_word_idx = y_test[i]
        predicted_prob = predictions[i][true_word_idx]
        # Evitar log(0) agregando un pequeño epsilon
        predicted_prob = max(predicted_prob, 1e-10)
        log_likelihoods.append(np.log(predicted_prob))
    
    # Calcular perplejidad
    average_log_likelihood = np.mean(log_likelihoods)
    perplexity = np.exp(-average_log_likelihood)
    
    return perplexity

def interpret_perplexity(perplexity_value):
    """
    Interpreta el valor de perplejidad según rangos comunes.
    
    Args:
        perplexity_value (float): Valor de perplejidad calculado
        
    Returns:
        str: Interpretación del valor
    """
    if perplexity_value < 10:
        return "Excelente"
    elif perplexity_value < 50:
        return "Muy bueno"
    elif perplexity_value < 100:
        return "Bueno"
    elif perplexity_value < 200:
        return "Aceptable"
    elif perplexity_value < 500:
        return "Regular"
    else:
        return "Pobre"

## 10. Evaluación Completa del Modelo

Se evalua el rendimiento del modelo incluyendo perplejidad y métricas estándar.

In [ ]:
def evaluate_model_performance(history, model, X_test, y_test):
    """
    Muestra métricas de rendimiento del modelo durante el entrenamiento y evaluación.
    """
    print("\n=== MÉTRICAS DE ENTRENAMIENTO ===")
    final_loss = history.history['loss'][-1]
    final_accuracy = history.history['accuracy'][-1]
    
    if 'val_loss' in history.history:
        final_val_loss = history.history['val_loss'][-1]
        final_val_accuracy = history.history['val_accuracy'][-1]
        print(f"Loss final: {final_loss:.4f} | Val Loss: {final_val_loss:.4f}")
        print(f"Accuracy final: {final_accuracy:.4f} | Val Accuracy: {final_val_accuracy:.4f}")
    else:
        print(f"Loss final: {final_loss:.4f}")
        print(f"Accuracy final: {final_accuracy:.4f}")
    
    # Evaluar en conjunto de prueba
    print("\n=== EVALUACIÓN EN CONJUNTO DE PRUEBA ===")
    test_results = model.evaluate(X_test, y_test, verbose=0)
    test_loss = test_results[0]
    test_accuracy = test_results[1]
    print(f"Test Loss: {test_loss:.4f}")
    print(f"Test Accuracy: {test_accuracy:.4f}")
    
    # Si hay métricas adicionales, mostrarlas también
    if len(test_results) > 2:
        test_top_k_accuracy = test_results[2]
        print(f"Test Top-K Accuracy: {test_top_k_accuracy:.4f}")
    
    # Calcular perplejidad
    print("\n=== CÁLCULO DE PERPLEJIDAD ===")
    perplexity = calculate_perplexity(model, X_test, y_test)
    interpretation = interpret_perplexity(perplexity)
    
    print(f"Perplejidad en conjunto de prueba: {perplexity:.2f}")
    print(f"Interpretación: {interpretation}")
    
    # Tabla de interpretación
    print("\n=== TABLA DE INTERPRETACIÓN DE PERPLEJIDAD ===")
    print("< 10:     Excelente")
    print("10-50:    Muy bueno") 
    print("50-100:   Bueno")
    print("100-200:  Aceptable")
    print("200-500:  Regular")
    print("> 500:    Pobre")
    
    return perplexity

# Evaluar rendimiento (incluyendo perplejidad)
perplexity = evaluate_model_performance(history, model, X_test, y_test)

## 11. Predicción de la Próxima Palabra

Se implementa la función `predict_next_word` para predecir la siguiente palabra dada una secuencia de entrada.

In [ ]:
def predict_next_word(model, tokenizer, sentence, max_length, top_k=1):
    """
    Predice la(s) palabra(s) siguiente(s) dada una oración en español.
    
    Args:
        model: Modelo entrenado
        tokenizer: Tokenizador ajustado
        sentence (str): Oración de entrada
        max_length (int): Longitud máxima de secuencia
        top_k (int): Número de predicciones top a retornar
    
    Returns:
        str o list: Palabra predicha (top_k=1) o lista de predicciones (top_k>1)
    """
    # Tokenizar la oración
    sequence = tokenizer.texts_to_sequences([sentence])[0]

    if len(sequence) == 0:
        return "<no_reconocido>" if top_k == 1 else ["<no_reconocido>"]

    # Aplicar padding usando el mismo tipo configurado
    sequence_padded = pad_sequences([sequence], maxlen=max_length, padding=PADDING_TYPE, truncating='pre')

    # Predecir probabilidades
    prediction = model.predict(sequence_padded, verbose=VERBOSE_PREDICTION)
    
    if top_k == 1:
        # Retornar solo la predicción más probable
        predicted_idx = np.argmax(prediction[0])
        predicted_word = tokenizer.index_word.get(predicted_idx, "<desconocido>")
        return predicted_word
    else:
        # Retornar las top_k predicciones más probables
        top_indices = np.argsort(prediction[0])[-top_k:][::-1]
        predictions = []
        for idx in top_indices:
            word = tokenizer.index_word.get(idx, "<desconocido>")
            prob = prediction[0][idx]
            predictions.append((word, prob))
        return predictions

def generate_text(model, tokenizer, seed_text, max_length, num_words_to_generate=10):
    """
    Genera texto continuando desde un texto semilla.
    
    Args:
        model: Modelo entrenado
        tokenizer: Tokenizador ajustado
        seed_text (str): Texto inicial
        max_length (int): Longitud máxima de secuencia
        num_words_to_generate (int): Número de palabras a generar
    
    Returns:
        str: Texto generado
    """
    generated_text = seed_text
    
    for _ in range(num_words_to_generate):
        next_word = predict_next_word(model, tokenizer, generated_text, max_length)
        if next_word in ["<no_reconocido>", "<desconocido>"]:
            break
        generated_text += " " + next_word
    
    return generated_text

print("Funciones de predicción definidas exitosamente!")

## 12. Pruebas de Predicción de Siguiente Palabra

Se prueba el modelo con casos de prueba en español.

In [ ]:
# Casos de prueba para predicción de siguiente palabra
test_cases = [
    "el gato se sentó en la",
    "los estudiantes abrieron sus",
    "la maestra escribió en el",
    "el niño jugó con un",
    "el pájaro voló sobre el",
    "el sol se elevó en el",
    "la casa tiene una",
    "me gusta comer",
    "vamos a la"
]

print(f"=== PREDICCIÓN DE SIGUIENTE PALABRA ({'BiLSTM' if USE_BIDIRECTIONAL else 'LSTM'}) ===")
for sentence in test_cases:
    next_word = predict_next_word(model, tokenizer, sentence, MAX_LEN)
    print(f"'{sentence}' → '{next_word}'")

## 13. Predicciones Top-K

Se muestran las K(3) predicciones más probables para algunos casos.

In [ ]:
# Predicciones top-k para algunos casos
print(f"=== TOP-3 PREDICCIONES ===")
for sentence in test_cases[:3]:
    top_predictions = predict_next_word(model, tokenizer, sentence, MAX_LEN, top_k=3)
    print(f"'{sentence}':")
    for i, (word, prob) in enumerate(top_predictions, 1):
        print(f"  {i}. '{word}' (probabilidad: {prob:.4f})")
    print()

## 14. Generación de Texto

Se genera texto continuando desde un texto semilla.

In [ ]:
# Generación de texto
print(f"=== GENERACIÓN DE TEXTO ===")
seed_texts = [
    "el perro",
    "los estudiantes",
    "en la casa"
]

for seed in seed_texts:
    generated = generate_text(model, tokenizer, seed, MAX_LEN, num_words_to_generate=8)
    print(f"Semilla: '{seed}'")
    print(f"Generado: '{generated}'")
    print()

## 15. Resumen Final del Modelo

Se muestra un resumen completo de las características y rendimiento del modelo.

In [ ]:
print("=== RESUMEN DEL MODELO ===")
print(f"Vocabulario: {vocab_size:,} palabras")
print(f"Secuencias de entrenamiento: {len(X_train):,}")
print(f"Secuencias de prueba: {len(X_test):,}")
print(f"Longitud máxima de secuencia (MAX_LEN): {MAX_LEN}")
print(f"Arquitectura: {'BiLSTM' if USE_BIDIRECTIONAL else 'LSTM'}")
print(f"Parámetros del modelo: {model.count_params():,}")
print(f"Dataset utilizado: {DATASET_NAME} (primeras {DATASET_TAKE:,} muestras)")
print(f"Perplejidad final: {perplexity:.2f} ({interpret_perplexity(perplexity)})")

print("\n=== COMPONENTES IMPLEMENTADOS ===")
components = [
    "Carga del Dataset (spanish_billion_words_clean)",
    "Tokenización y Creación del Vocabulario", 
    "Creación del Conjunto de Entrenamiento (X, Y)",
    "Padding y Truncado (MAX_LEN ≤ 50)",
    "División del Conjunto (Train/Test 80%/20%)",
    "Construcción del Modelo LSTM/BiLSTM",
    "Entrenamiento con Early Stopping",
    "Cálculo de la Perplejidad",
    "Predicción de la Próxima Palabra",
    "Uso de sparse_categorical_crossentropy"
]

for component in components:
    print(component)

---

## Conclusiones

Este notebook implementa completamente los requerimientos de la Tarea 1:

1. **Dataset**: Se utilizó `spanish_billion_words_clean` de Hugging Face
2. **Tokenización**: Se Implementó con vocabulario único y mapeo palabra↔índice
3. **Preparación de datos**: Secuencias (X,Y) con padding/truncado (MAX_LEN≤50)
4. **División**: 80% entrenamiento / 20% prueba con `train_test_split`
5. **Modelo**: Arquitectura LSTM/BiLSTM configurable con embedding
6. **Entrenamiento**: Con Early Stopping y `sparse_categorical_crossentropy`
7. **Evaluación**: Perplejidad calculada 
8. **Predicción**: Función `predict_next_word`

El modelo puede alternar entre LSTM y BiLSTM simplemente cambiando `USE_BIDIRECTIONAL=True/False` en los parámetros globales.

**Parámetros recomendados para experimentación:**
- EMBEDDING_DIM: 100-300
- LSTM_UNITS: 64-128
- DENSE_UNITS: 64-128
- LEARNING_RATE: 0.001-0.01
- BATCH_SIZE: 8-32